In [ ]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

%load_ext google.colab.data_table

Authenticated


User Lifetime Value - to bardzo abstrakcyjny wskaźnik KPI. Musisz zdecydować, jaki poziom abstrakcji użyjesz w swoich analizach, tak aby odpowiadał oczekiwaniom Twojej firmy lub organizacji względem tego wskaźnika.

Przychody i utrzymanie idą w parze. Im większe prawdopodobieństwo, że użytkownicy powrócą do Twojej witryny, tym wyższy dochód wygenerowany przez tych użytkowników w dłuższej perspektywie.

Calculate User Lifetime Revenue

In [ ]:
%%bigquery --project produkcja-instore

select
    user_pseudo_id,
    sum(ecommerce.purchase_revenue) as lifetime_value
from
    `simoahava-com.analytics_206575074.events_*`
where
    _table_suffix between format_date('%Y%m%d',date_sub(current_date(), interval 60 day))
    and format_date('%Y%m%d',date_sub(current_date(), interval 1 day))
    and ecommerce.purchase_revenue is not null
group by 
    user_pseudo_id
order by 
    lifetime_value desc
LIMIT 5

,user_pseudo_id,lifetime_value
0,1612950054.1622107524,2612.0
1,659974159.1624173024,2490.0
2,1363679883.1623167021,2441.0
3,109766058.1623347678,2410.0
4,1684845558.1612992021,2285.0


Calculate Average Lifetime Revenue

In [ ]:
%%bigquery --project produkcja-instore

with prep as (
select
    user_pseudo_id,
    (select value.int_value from unnest(event_params) where key = 'ga_session_id') as session_id,
    parse_date('%Y%m%d',event_date) as session_date,
    first_value(parse_date('%Y%m%d',event_date)) over (partition by user_pseudo_id order by event_date) as first_session_date,
    sum(ecommerce.purchase_revenue) as revenue
from
    `simoahava-com.analytics_206575074.events_*`
where
    _table_suffix between format_date('%Y%m%d',date_sub(current_date(), interval 60 day)) and format_date('%Y%m%d',date_sub(current_date(), interval 1 day))
group by
    user_pseudo_id,
    session_id,
    event_date
order by
    user_pseudo_id,
    session_id),

prep_ltv as (
select 
    date_diff(session_date,first_session_date,day) as day,
    count(distinct user_pseudo_id) as users,
    sum(revenue) as ltv_revenue_per_day,
    sum(revenue) / max(count(distinct user_pseudo_id)) over () as avg_ltv_revenue_per_day
from 
    prep
group by 
    day
order by 
    day)

select 
    day,
    sum(avg_ltv_revenue_per_day) over (order by day) as average_ltv_revenue
from 
    prep_ltv
group by 
    day,
    avg_ltv_revenue_per_day
order by 
    day

LIMIT 5

,day,average_ltv_revenue
0,0,14.423031
1,1,16.036773
2,2,16.728057
3,3,17.295210
4,4,17.773599
